In [72]:
import pandas as pd
import math


In [73]:
def base_entropy(dataset):
    p = 0
    n = 0
    target = dataset.iloc[:, 0] # get the target column
    targets = list(set(target)) # set is used to get the categorical values, and use list to make an array
    for i in target:
        if i == targets[0]:
            p = p + 1
        else:
            n = n + 1
    if p == 0 or n == 0: # if one of it has already been divided
        return 0
    elif p == n: # if all the categories have equal chances - entropy = 1
        return 1
    else:
        entropy = 0 - (
            ((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy


In [74]:
def entropy(dataset, feature, attribute):
    p = 0
    n = 0
    target = dataset.iloc[:, 0]
    targets = list(set(target))
    for i, j in zip(feature, target):
        if i == attribute and j == targets[0]:
            p = p + 1
        elif i == attribute and j == targets[1]:
            n = n + 1
    if p == 0 or n == 0:
        return 0
    elif p == n:
        return 1
    else:
        entropy = 0 - (
            ((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy

In [75]:
def counter(target, attribute, i):
    p = 0
    n = 0
    targets = list(set(target))
    for j, k in zip(target, attribute):
        if j == targets[0] and k == i:
            p = p + 1
        elif j == targets[1] and k == i:
            n = n + 1
    return p, n

In [76]:
def Information_Gain(dataset, feature):
    Distinct = list(set(feature))
    Info_Gain = 0
    for i in Distinct:
        Info_Gain = Info_Gain + feature.count(i) / len(feature) * entropy(dataset, feature, i)
    Info_Gain = base_entropy(dataset) - Info_Gain
    return Info_Gain

In [77]:
def generate_childs(dataset, attribute_index):
    distinct = list(dataset.iloc[:, attribute_index])
    childs = dict()
    for i in distinct:
        childs[i] = counter(dataset.iloc[:, 0], dataset.iloc[:, attribute_index], i)
    return childs

In [78]:
def modify_data_set(dataset,index, feature, impurity):
    size = len(dataset)
    subdata = dataset[dataset[feature] == impurity]
    del (subdata[subdata.columns[index]])
    return subdata

In [79]:
def greatest_information_gain(dataset):
    max = -1
    attribute_index = 0
    size = len(dataset.columns) - 1
    for i in range(1, size):
        feature = list(dataset.iloc[:, i]) # take each column
        i_g = Information_Gain(dataset, feature) # check the information gain of each column
        if max < i_g:
            max = i_g
            attribute_index = i
    return attribute_index # return the column with max information gain

In [80]:
def construct_tree(dataset, tree):
    target = dataset.iloc[:, 0]
    impure_childs = []
    attribute_index = greatest_information_gain(dataset) # find the first node?
    childs = generate_childs(dataset, attribute_index)
    tree[dataset.columns[attribute_index]] = childs
    targets = list(set(dataset.iloc[:, 0]))
    for k, v in childs.items():
        if v[0] == 0:
            tree[k] = targets[1]
        elif v[1] == 0:
            tree[k] = targets[0]
        elif v[0] != 0 or v[1] != 0:
            impure_childs.append(k)
    for i in impure_childs:
        sub = modify_data_set(dataset,attribute_index, dataset.columns[attribute_index], i)
        tree = construct_tree(sub, tree)
    return tree

In [81]:
def main():
    df = pd.read_csv("agaricus-lepiota.data", header = None)
    tree = dict()
    result = construct_tree(df, tree)
    for key, value in result.items():
        print(key, " => ", value)

In [82]:
if __name__ == "__main__":
    main()

5  =>  {'p': (0, 256), 'a': (400, 0), 'l': (400, 0), 'n': (3408, 120), 'f': (0, 2160), 'c': (0, 192), 'y': (0, 576), 's': (0, 576), 'm': (0, 36)}
p  =>  p
a  =>  e
l  =>  e
f  =>  e
c  =>  e
y  =>  p
s  =>  p
m  =>  p
20  =>  {'n': (1344, 0), 'k': (1296, 0), 'w': (576, 48), 'h': (48, 0), 'r': (0, 72), 'o': (48, 0), 'y': (48, 0), 'b': (48, 0)}
n  =>  e
k  =>  p
h  =>  e
r  =>  p
o  =>  e
b  =>  e
8  =>  {'b': (528, 0), 'n': (48, 48)}
12  =>  {'f': (24, 0), 'k': (0, 32), 's': (24, 8), 'y': (0, 8)}
3  =>  {'w': (0, 8), 'c': (12, 0), 'n': (12, 0)}
w  =>  p


In [83]:
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
